In [ ]:
import os
import yt_dlp
import numpy as np
import soundfile as sf
import pyloudnorm as pyln
from openpyxl import load_workbook
from pydub import AudioSegment
from mutagen.wave import WAVE

# === CONFIG ===
EXPORT_FOLDER = "procuced_wav_files"
TARGET_LUFS = -14.0
os.makedirs(EXPORT_FOLDER, exist_ok=True)

# Load song titles
def load_song_titles(xlsx_path):
    wb = load_workbook(xlsx_path)
    ws = wb.active
    return [row[0].value for row in ws.iter_rows(min_row=2) if row[0].value]

# Download highest quality audio
def download_best_audio(song_title):
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': True,
        'default_search': 'ytsearch',
        'outtmpl': f'{EXPORT_FOLDER}/{song_title}.%(ext)s',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print(f"🔍 Downloading: {song_title}")
        try:
            ydl.download([song_title])
        except Exception as e:
            print(f"❌ Failed to download '{song_title}': {e}")

# Convert to WAV and apply LUFS normalization
def convert_to_lufs_wav(song_title):
    raw_path = None
    for ext in ["webm", "m4a", "opus", "mp3"]:
        candidate = os.path.join(EXPORT_FOLDER, f"{song_title}.{ext}")
        if os.path.exists(candidate):
            raw_path = candidate
            break

    if not raw_path:
        print(f"❌ No downloaded audio found for: {song_title}")
        return

    print(f"🎼 Converting and normalizing: {song_title}")
    audio = AudioSegment.from_file(raw_path)
    wav_temp_path = os.path.join(EXPORT_FOLDER, f"{song_title}_temp.wav")
    audio.export(wav_temp_path, format="wav")

    # Load raw WAV into numpy
    y, sr = sf.read(wav_temp_path)
    if len(y.shape) > 1:  # stereo to mono
        y = y.mean(axis=1)

    # Measure loudness
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(y)
    print(f"📏 Original LUFS: {loudness:.2f}")

    # Normalize
    y_normalized = pyln.normalize.loudness(y, loudness, TARGET_LUFS)

    # Export normalized audio
    final_wav_path = os.path.join(EXPORT_FOLDER, f"{song_title}.wav")
    sf.write(final_wav_path, y_normalized, sr)
    os.remove(wav_temp_path)
    os.remove(raw_path)
    print(f"✅ LUFS-normalized WAV saved: {final_wav_path}")

    # Optional metadata
    add_metadata(final_wav_path, song_title)

# Add metadata to WAV
def add_metadata(filepath, song_title):
    artist = "Unknown Artist"
    if " - " in song_title:
        artist, song_title = song_title.split(" - ", 1)

    try:
        audio = WAVE(filepath)
        audio["title"] = song_title
        audio["artist"] = artist
        audio.save()
        print(f"🏷️ Metadata tagged: {song_title} by {artist}")
    except Exception as e:
        print(f"⚠️ Metadata tagging failed: {e}")

# Main
if __name__ == "__main__":
    titles = load_song_titles("songs.xlsx")
    for title in titles:
        download_best_audio(title)
        convert_to_lufs_wav(title)


🔍 Downloading: Dio Sacred Heart
🎼 Converting and normalizing: Dio Sacred Heart          
📏 Original LUFS: -12.46
✅ LUFS-normalized WAV saved: lufs_audio\Dio Sacred Heart.wav
⚠️ Metadata tagging failed: 'Dio Sacred Heart' not a Frame instance
🔍 Downloading: The Tubes She's a Beauty
🎼 Converting and normalizing: The Tubes She's a Beauty  
📏 Original LUFS: -16.55
✅ LUFS-normalized WAV saved: lufs_audio\The Tubes She's a Beauty.wav
⚠️ Metadata tagging failed: "The Tubes She's a Beauty" not a Frame instance


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


🔍 Downloading: Deep Purple The king of Dreams
🎼 Converting and normalizing: Deep Purple The king of Dreams
📏 Original LUFS: -13.56
✅ LUFS-normalized WAV saved: lufs_audio\Deep Purple The king of Dreams.wav
⚠️ Metadata tagging failed: 'Deep Purple The king of Dreams' not a Frame instance
🔍 Downloading: Crazy Lixx One Fire One Goal
🎼 Converting and normalizing: Crazy Lixx One Fire One Goal
📏 Original LUFS: -16.14


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: lufs_audio\Crazy Lixx One Fire One Goal.wav
⚠️ Metadata tagging failed: 'Crazy Lixx One Fire One Goal' not a Frame instance
🔍 Downloading: Grave Digger Rebellion 
🎼 Converting and normalizing: Grave Digger Rebellion    
📏 Original LUFS: -21.85
✅ LUFS-normalized WAV saved: lufs_audio\Grave Digger Rebellion .wav
⚠️ Metadata tagging failed: 'Grave Digger Rebellion ' not a Frame instance
🔍 Downloading: Tyketto Forever Young
🎼 Converting and normalizing: Tyketto Forever Young     
📏 Original LUFS: -20.71


...

🔍 Downloading: Firehouse All she Wrote
🎼 Converting and normalizing: Firehouse All she Wrote   
📏 Original LUFS: -17.57
✅ LUFS-normalized WAV saved: lufs_audio\Firehouse All she Wrote.wav
⚠️ Metadata tagging failed: 'Firehouse All she Wrote' not a Frame instance


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


🔍 Downloading: Accept Midnight Mover
🎼 Converting and normalizing: Accept Midnight Mover     
📏 Original LUFS: -17.74
✅ LUFS-normalized WAV saved: lufs_audio\Accept Midnight Mover.wav
⚠️ Metadata tagging failed: 'Accept Midnight Mover' not a Frame instance


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


🔍 Downloading: Warlock All We Are
🎼 Converting and normalizing: Warlock All We Are        
